# Preparation

## Import Packages

In [118]:
import os
import re

import pandas as pd
import requests
from requests.exceptions import ConnectionError, ReadTimeout
from requests.exceptions import ContentDecodingError, TooManyRedirects

from db.mongo import MyMongo

## Load File

In [3]:
data_dir = '/Users/jake/OneDrive - leverage innovative users/Documents/News_Item/Audit_quote/'
quote_file = os.path.join(data_dir, 'quotation_all.tsv')

quote = pd.read_csv(quote_file, sep='\t', dtype=object)
quote.head()

,뉴스 식별자,일자,언론사,정보원,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인용문,키워드,특성추출,분석제외 여부
0,02100701.20181030084120001,20181030,헤럴드경제,이 의원,미분류,NaN,NaN,NaN,NaN,NaN,"현재 외국인 관광객의 80% 정도는 서울 및 수도권에 몰리는 반면, 타 지방을 방문...","외국인,관광객,정도,서울,수도권,지방,방문,외국인,관광객,수준,실질,규제,혁신,제도...","관광객,관광인프라,외국인_관광객,서울,제대로,외국인,제도개선,수도권,실질적_규제혁신...",NaN
1,02100701.20181030081315001,20181030,헤럴드경제,김병욱 의원,미분류,NaN,NaN,NaN,NaN,NaN,청년실업 등으로 신용등급이 낮고 형편이 어려운 20대가 은행권에서 대출을 받지 못하...,"청년실업,신용등급,형편,20대,은행,대출,금리,대부업체,상환,능력,연체율,증가,","신용등급,연체율,20대,은행권,상환능력,대부업체,청년실업,형편,능력,상환,은행,대출...",NaN
2,01500801.20181030072457002,20181030,영남일보,자유한국당 강효상 의원,사회>환경,지역>대구,NaN,NaN,NaN,NaN,지난 6월29일 물산업클러스터 위탁운영기관으로 한국환경공단이 한국수자원공사를 제치고...,"6월,29일,물산업클러스터,위탁,운영,기관,한국환경공단,한국수자원공사,선정,배경,환...","물산업클러스터,환경부,한국수자원공사,한국환경공단,물산업클러스터_위탁운영기관,배경,선...",NaN
3,08100201.20181030064343002,20181030,MBC,김재경 자유한국당 의원,정치>국회_정당,정치>외교,정치>북한,NaN,NaN,NaN,하나는 비준 동의를 해야 하고 하나는 안 해야 하는 차별성이 있는 것이냐,"하나,비준,동의,하나,차별성,","비준_동의,차별성,동의,비준",NaN
4,01100701.20181030060258001,20181030,세계일보,더불어민주당 전혜숙 의원,사회>사건_사고,지역>경북,사회>미디어,NaN,범죄>범죄일반>절도,NaN,"가정의학과 의사의 경우 선한 의지로 응급진료를 도운 것뿐인데, 오히려 가해자가 된 ...","가정의학과,의사,의지,응급,진료,가해자,사마리아인,면책,응급,상황,환자,","사마리아인,사마리아인들,가해자,응급진료,응급상황,가정의학과_의사,가정의학과,응급,의...",NaN


In [80]:
quote['url'] ="https://www.bigkinds.or.kr/news/detailView.do?docId="+quote['뉴스 식별자']+"&returnCnt=1&sectionDiv=1000&indexName="

## 의원 명단

In [6]:
members = ['강길부', '강병원', '강석진', '강석호', '강창일', '강효상', '강훈식', '경대수', '고용진', '곽대훈', '곽상도', '권미혁', '권성동', '권은희', '권칠승', '금태섭', '기동민', '김경진', '김경협', '김관영', '김광림', '김광수', '김규환', '김기선', '김도읍', '김동철', '김두관', '김명연', '김무성', '김민기', '김병관', '김병기', '김병욱', '김부겸', '김삼화', '김상훈', '김상희', '김석기', '김선동', '김성수', '김성식', '김성원', '김성찬', '김성태', '김성태', '김성환', '김세연', '김수민', '김순례', '김승희', '김영우', '김영주', '김영진', '김영춘', '김영호', '김용태', '김재경', '김재원', '김정우', '김정재', '김정호', '김정훈', '김종대', '김종민', '김종석', '김종회', '김종훈', '김중로', '김진태', '김진표', '김철민', '김태년', '김태흠', '김학용', '김한정', '김한표', '김해영', '김현권', '김현미', '김현아', '나경원', '남인순', '노웅래', '도종환', '맹성규', '문진국', '문희상', '민경욱', '민병두', '민홍철', '박경미', '박광온', '박대출', '박덕흠', '박맹우', '박명재', '박범계', '박병석', '박선숙', '박성중', '박순자', '박영선', '박완수', '박완주', '박용진', '박인숙', '박재호', '박정', '박주민', '박주선', '박주현', '박지원', '박찬대', '박홍근', '백승주', '백재현', '백혜련', '변재일', '서삼석', '서영교', '서청원', '서형수', '설훈', '성일종', '소병훈', '손금주', '손혜원', '송갑석', '송기헌', '송석준', '송언석', '송영길', '송옥주', '송희경', '신경민', '신동근', '신보라', '신상진', '신용현', '신창현', '심기준', '심상정', '심재권', '심재철', '안규백', '안민석', '안상수', '안호영', '어기구', '엄용수', '여상규', '염동열', '오신환', '오영훈', '오제세', '우상호', '우원식', '원유철', '원혜영', '위성곤', '유기준', '유동수', '유민봉', '유성엽', '유승민', '유승희', '유은혜', '유의동', '유재중', '윤관석', '윤상직', '윤상현', '윤소하', '윤영석', '윤영일', '윤일규', '윤재옥', '윤종필', '윤준호', '윤한홍', '윤호중', '윤후덕', '이개호', '이군현', '이규희', '이동섭', '이만희', '이명수', '이상돈', '이상민', '이상헌', '이석현', '이수혁', '이양수', '이언주', '이완영', '이용득', '이용주', '이용호', '이우현', '이원욱', '이은권', '이은재', '이인영', '이장우', '이재정', '이정미', '이정현', '이종걸', '이종구', '이종명', '이종배', '이주영', '이진복', '이찬열', '이채익', '이철규', '이철희', '이춘석', '이태규', '이학영', '이학재', '이해찬', '이헌승', '이현재', '이혜훈', '이후삼', '이훈', '인재근', '임이자', '임재훈', '임종성', '장병완', '장석춘', '장정숙', '장제원', '전재수', '전해철', '전현희', '전혜숙', '전희경', '정갑윤', '정동영', '정병국', '정성호', '정세균', '정양석', '정용기', '정우택', '정운천', '정유섭', '정인화', '정재호', '정종섭', '정진석', '정춘숙', '정태옥', '제윤경', '조경태', '조배숙', '조승래', '조원진', '조응천', '조정식', '조훈현', '주광덕', '주승용', '주호영', '지상욱', '진선미', '진영', '채이배', '천정배', '최경환', '최경환', '최교일', '최도자', '최연혜', '최운열', '최인호', '최재성', '추경호', '추미애', '추혜선', '표창원', '하태경', '한선교', '한정애', '함진규', '홍문종', '홍문표', '홍영표', '홍의락', '홍익표', '홍일표', '홍철호', '황영철', '황주홍', '황희']

members2 = []  # 박영 선 case
members3 = []  # 박 영선 case
for m in members:
    if len(m) == 3:
        members2.append(m[0:2] + ' ' + m[2])
        members3.append(m[0:1] + ' ' + m[1:3])
    else:
        members2.append(m)
        members3.append(m)

# Transform

## '의원' 제거

In [59]:
def extract_name(x):
    try:
        return re.search(r'(.+)의원', x).group(1).strip()
    except AttributeError as e:
        return ''

quote['정보원_추출'] = quote['정보원'].apply(extract_name)

## 길이 3 이상

In [60]:
idx_gt_3 = quote['정보원_추출'].str.len() >= 3; idx_gt_3

0       False
1        True
2        True
3        True
4        True
5       False
6        True
7        True
8        True
9       False
10       True
11      False
12       True
13      False
14      False
15      False
16       True
17      False
18      False
19       True
20       True
21       True
22      False
23       True
24       True
25       True
26       True
27       True
28      False
29      False
        ...  
8420    False
8421    False
8422    False
8423    False
8424    False
8425     True
8426     True
8427     True
8428     True
8429     True
8430    False
8431    False
8432    False
8433     True
8434    False
8435     True
8436     True
8437     True
8438     True
8439    False
8440    False
8441    False
8442     True
8443     True
8444    False
8445     True
8446     True
8447     True
8448     True
8449    False
Name: 정보원_추출, Length: 8450, dtype: bool

## 길이 2 이하

In [61]:
idx_lt_2 = quote['정보원_추출'].str.len() <= 2; idx_lt_2

0        True
1       False
2       False
3       False
4       False
5        True
6       False
7       False
8       False
9        True
10      False
11       True
12      False
13       True
14       True
15       True
16      False
17       True
18       True
19      False
20      False
21      False
22       True
23      False
24      False
25      False
26      False
27      False
28       True
29       True
        ...  
8420     True
8421     True
8422     True
8423     True
8424     True
8425    False
8426    False
8427    False
8428    False
8429    False
8430     True
8431     True
8432     True
8433    False
8434     True
8435    False
8436    False
8437    False
8438    False
8439     True
8440     True
8441     True
8442    False
8443    False
8444     True
8445    False
8446    False
8447    False
8448    False
8449     True
Name: 정보원_추출, Length: 8450, dtype: bool

## 식별된 의원 & 비식별 의원

In [99]:
recognized = []

for i, row in quote.iterrows():
    value = row['정보원_추출']
    flag= False
    recognized_name = ''
    for m in members:
        if m in value:
            recognized.append(True)
            flag = True
            recognized_name = m
            break
    if not flag:
        for m2 in members2:
            if m2 in value:
                recognized.append(True)
                flag = True
                recognized_name = m
                break
    if not flag:
        for m3 in members3:
            if m3 in value:
                recognized.append(True)
                flag = True
                recognized_name = m
                break
    if flag:
        quote.at[i, '정보원_추출2'] = recognized_name
    else:
        recognized.append(False)

idx_recognized = pd.Series(recognized)
idx_unknown = ~idx_recognized

In [90]:
print(len(quote))
print(len(recognized))
print(len(quote.loc[idx_recognized]))
print(len(quote.loc[idx_unknown]))

8450
8450
5371
3079


In [81]:
quote.loc[idx_unknown, ['뉴스 식별자', '정보원', '정보원_추출', 'url']]

,뉴스 식별자,정보원,정보원_추출,url
0,02100701.20181030084120001,이 의원,이,https://www.bigkinds.or.kr/news/detailView.do?...
5,01100701.20181030060141002,의원,,https://www.bigkinds.or.kr/news/detailView.do?...
9,01101101.20181030051243002,의원 보좌관,,https://www.bigkinds.or.kr/news/detailView.do?...
11,02100201.20181030043229001,박 의원,박,https://www.bigkinds.or.kr/news/detailView.do?...
13,01100201.20181030040156001,의원,,https://www.bigkinds.or.kr/news/detailView.do?...
14,01100201.20181030040156001,한 의원,한,https://www.bigkinds.or.kr/news/detailView.do?...
15,01100201.20181030040154001,의원,,https://www.bigkinds.or.kr/news/detailView.do?...
16,01100201.20181030040154001,경북도의회 박승직 경주 의원,경북도의회 박승직 경주,https://www.bigkinds.or.kr/news/detailView.do?...
17,01100611.20181030034115001,김 의원,김,https://www.bigkinds.or.kr/news/detailView.do?...
18,01100611.20181030034050001,박 의원,박,https://www.bigkinds.or.kr/news/detailView.do?...


In [66]:
quote.loc[5, ['정보원_추출']].str.len()

정보원_추출    0
Name: 5, dtype: int64

## '정보원_추출' 길이 1 (의원 성)

In [73]:
idx_len_1 = quote['정보원_추출'].str.len() == 1

In [89]:
len([i for i in idx_len_1 if i==True])

2498

## '정보원_추출2' 채우기

In [91]:
# 인식된 의원('박범계 의원' 등)과 '박 의원' 집합 사이에 교집합이 있는지 체크
(idx_recognized & idx_len_1).any()

False

### 이 의원, 박 의원 변환

In [97]:
def filter_refer(x, first_character):
    rexp = '^(' + re.escape(first_character) + ').+'
    
    return len(x) >= 2 and re.search(rexp, x)

idx_error = []
requests_error = []

for i, row in quote.loc[idx_len_1, ['뉴스 식별자', '정보원_추출']].iterrows():
#     if i == 14:
#         break
    news_id = row['뉴스 식별자']
    url = f'https://www.bigkinds.or.kr/news/detailView.do?docId={news_id}&returnCnt=1&sectionDiv=1000&indexName='
    
    try:
        response = requests.get(url, timeout=3)

    except ReadTimeout:
        requests_error.append(i)
        continue
    except ContentDecodingError:
        requests_error.append(i)
        continue
    except TooManyRedirects:
        requests_error.append(i)
        continue
    
    if str(response.status_code)[0] == '4':
        status_error.append(i)
    people = response.json()['detail']['TMS_NE_PERSON']
    
    try:
        first_character = row['정보원_추출'][0]
    except IndexError as e:
        idx_error.append(i)
        continue
        
    people_list = list(filter(lambda x: filter_refer(x, first_character), people.split('\n')))
    people_only_legislator = [p for p in people_list if p in members]
    
    if len(people_only_legislator) == 1:
        result = people_only_legislator[0]
    else:
        result = ''

    quote.at[i, '정보원_추출2'] = result

In [104]:
pd.set_option('display.max_colwidth', -1)
quote[['정보원', '정보원_추출', '정보원_추출2', 'url']]

,정보원,정보원_추출,정보원_추출2,url
0,이 의원,이,이상헌,https://www.bigkinds.or.kr/news/detailView.do?docId=02100701.20181030084120001&returnCnt=1&sectionDiv=1000&indexName=
1,김병욱 의원,김병욱,김병욱,https://www.bigkinds.or.kr/news/detailView.do?docId=02100701.20181030081315001&returnCnt=1&sectionDiv=1000&indexName=
2,자유한국당 강효상 의원,자유한국당 강효상,강효상,https://www.bigkinds.or.kr/news/detailView.do?docId=01500801.20181030072457002&returnCnt=1&sectionDiv=1000&indexName=
3,김재경 자유한국당 의원,김재경 자유한국당,김재경,https://www.bigkinds.or.kr/news/detailView.do?docId=08100201.20181030064343002&returnCnt=1&sectionDiv=1000&indexName=
4,더불어민주당 전혜숙 의원,더불어민주당 전혜숙,전혜숙,https://www.bigkinds.or.kr/news/detailView.do?docId=01100701.20181030060258001&returnCnt=1&sectionDiv=1000&indexName=
5,의원,,,https://www.bigkinds.or.kr/news/detailView.do?docId=01100701.20181030060141002&returnCnt=1&sectionDiv=1000&indexName=
6,자유한국당 장제원 의원,자유한국당 장제원,장제원,https://www.bigkinds.or.kr/news/detailView.do?docId=01100701.20181030060141002&returnCnt=1&sectionDiv=1000&indexName=
7,백재현 의원,백재현,백재현,https://www.bigkinds.or.kr/news/detailView.do?docId=02100201.20181030055140001&returnCnt=1&sectionDiv=1000&indexName=
8,백재현 더불어민주당 의원,백재현 더불어민주당,백재현,https://www.bigkinds.or.kr/news/detailView.do?docId=02100201.20181030055140001&returnCnt=1&sectionDiv=1000&indexName=
9,의원 보좌관,,,https://www.bigkinds.or.kr/news/detailView.do?docId=01101101.20181030051243002&returnCnt=1&sectionDiv=1000&indexName=


### fillna()

In [103]:
quote['정보원_추출2'] = quote['정보원_추출2'].fillna('')

### idx_not_empty

In [105]:
idx_not_empty = quote['정보원_추출2'] != ''

In [121]:
quote.loc[idx_not_empty, ['정보원', '정보원_추출', '정보원_추출2', 'url']]

/Users/jake/.virtualenvs/workspace/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,정보원,정보원_추출,정보원_추출2,url
0,이 의원,이,NaN,https://www.bigkinds.or.kr/news/detailView.do?docId=02100701.20181030084120001&returnCnt=1&sectionDiv=1000&indexName=
1,김병욱 의원,김병욱,NaN,https://www.bigkinds.or.kr/news/detailView.do?docId=02100701.20181030081315001&returnCnt=1&sectionDiv=1000&indexName=
2,자유한국당 강효상 의원,자유한국당 강효상,NaN,https://www.bigkinds.or.kr/news/detailView.do?docId=01500801.20181030072457002&returnCnt=1&sectionDiv=1000&indexName=
3,김재경 자유한국당 의원,김재경 자유한국당,NaN,https://www.bigkinds.or.kr/news/detailView.do?docId=08100201.20181030064343002&returnCnt=1&sectionDiv=1000&indexName=
4,더불어민주당 전혜숙 의원,더불어민주당 전혜숙,NaN,https://www.bigkinds.or.kr/news/detailView.do?docId=01100701.20181030060258001&returnCnt=1&sectionDiv=1000&indexName=
6,자유한국당 장제원 의원,자유한국당 장제원,NaN,https://www.bigkinds.or.kr/news/detailView.do?docId=01100701.20181030060141002&returnCnt=1&sectionDiv=1000&indexName=
7,백재현 의원,백재현,NaN,https://www.bigkinds.or.kr/news/detailView.do?docId=02100201.20181030055140001&returnCnt=1&sectionDiv=1000&indexName=
8,백재현 더불어민주당 의원,백재현 더불어민주당,NaN,https://www.bigkinds.or.kr/news/detailView.do?docId=02100201.20181030055140001&returnCnt=1&sectionDiv=1000&indexName=
10,더불어민주당 박주민 의원,더불어민주당 박주민,NaN,https://www.bigkinds.or.kr/news/detailView.do?docId=01101101.20181030051238002&returnCnt=1&sectionDiv=1000&indexName=
11,박 의원,박,NaN,https://www.bigkinds.or.kr/news/detailView.do?docId=02100201.20181030043229001&returnCnt=1&sectionDiv=1000&indexName=


### idx_empty

In [108]:
idx_empty = ~idx_not_empty

In [109]:
quote.loc[idx_empty, ['정보원', '정보원_추출', '정보원_추출2', 'url']]

,정보원,정보원_추출,정보원_추출2,url
5,의원,,,https://www.bigkinds.or.kr/news/detailView.do?docId=01100701.20181030060141002&returnCnt=1&sectionDiv=1000&indexName=
9,의원 보좌관,,,https://www.bigkinds.or.kr/news/detailView.do?docId=01101101.20181030051243002&returnCnt=1&sectionDiv=1000&indexName=
13,의원,,,https://www.bigkinds.or.kr/news/detailView.do?docId=01100201.20181030040156001&returnCnt=1&sectionDiv=1000&indexName=
14,한 의원,한,,https://www.bigkinds.or.kr/news/detailView.do?docId=01100201.20181030040156001&returnCnt=1&sectionDiv=1000&indexName=
15,의원,,,https://www.bigkinds.or.kr/news/detailView.do?docId=01100201.20181030040154001&returnCnt=1&sectionDiv=1000&indexName=
16,경북도의회 박승직 경주 의원,경북도의회 박승직 경주,,https://www.bigkinds.or.kr/news/detailView.do?docId=01100201.20181030040154001&returnCnt=1&sectionDiv=1000&indexName=
28,정 의원,정,,https://www.bigkinds.or.kr/news/detailView.do?docId=01400701.20181030010138004&returnCnt=1&sectionDiv=1000&indexName=
34,야당 의원,야당,,https://www.bigkinds.or.kr/news/detailView.do?docId=02100201.20181030003939001&returnCnt=1&sectionDiv=1000&indexName=
65,더불어민주당 의원,더불어민주당,,https://www.bigkinds.or.kr/news/detailView.do?docId=01200201.20181030223222003&returnCnt=1&sectionDiv=1000&indexName=
68,의원,,,https://www.bigkinds.or.kr/news/detailView.do?docId=01200201.20181030223211001&returnCnt=1&sectionDiv=1000&indexName=


## 정리

### 정보원_추출2 => 국회의원

In [112]:
quote = quote.rename(columns={'정보원_추출2': '국회의원'})
quote.columns

Index(['뉴스 식별자', '일자', '언론사', '정보원', '통합 분류1', '통합 분류2', '통합 분류3', '사건/사고 분류1',
       '사건/사고 분류2', '사건/사고 분류3', '인용문', '키워드', '특성추출', '분석제외 여부', '정보원_추출',
       '국회의원', 'url'],
      dtype='object')

In [124]:
quote.loc[idx_not_empty].groupby(['뉴스 식별자', '국회의원']).agg({'인용문':'count'})

인용문
뉴스 식별자                     국회의원     
01100101.20181010000707001 홍일표   1  
01100101.20181010110531001 김관영   1  
                           조배숙   1  
01100101.20181010111902001 홍철호   1  
01100101.20181010112506001 김도읍   1  
                           송기헌   1  
                           이은재   1  
                           이춘석   1  
01100101.20181010113126001 최인호   1  
01100101.20181010122140001 김진태   2  
01100101.20181010134353001 강훈식   1  
                           윤관석   1  
                           이현재   1  
01100101.20181010135917001 김경진   1  
                           박선숙   1  
                           이상민   1  
01100101.20181010152055001 백혜련   1  
                           이춘석   1  
                           주광덕   1  
01100101.20181010160557001 이태규   1  
                           추혜선   1  
01100101.20181010162612001 김진태   2  
01100101.20181010163743001 송희경   1  
01100101.20181010164157001 김광수   1  
                           신상진   1  
                           유재중   1  
01100101.20181010165338001 박지원   1  
01100101.20181010165851001 손혜원   1  
01100101.20181010172644001 김도읍   1  
                           백혜련   1  
...                             ..  
08100201.20181030064343002 김재경   1  
08100301.20181012171343001 백승주   1  
08100301.20181016181528002 정갑윤   1  
08100301.20181028212240001 하태경   1  
08100401.20181010172402001 이재정   1  
08100401.20181010181202001 김병욱   1  
08100401.20181011174802001 이상돈   1  
08100401.20181011180402001 손혜원   1  
08100401.20181012002003001 이상돈   1  
08100401.20181012180805003 백재현   1  
08100401.20181016175602002 김수민   1  
08100401.20181018135602001 박용진   2  
08100401.20181018143850001 김영우   1  
                           유민봉   1  
08100401.20181018145448003 금태섭   1  
                           이은재   1  
08100401.20181022112003002 전현희   1  
08100401.20181023134402001 박지원   1  
08200101.20181010121509002 김성수   1  
                           김성태   1  
                           정용기   1  
08200101.20181010210750003 소병훈   1  
08200101.20181010210755003 이철희   1  
08200101.20181010212314001 김종훈   1  
08200101.20181010212314002 정용기   1  
08200101.20181011181050001 김무성   1  
08200101.20181012180019001 장제원   1  
                           조응천   1  
08200101.20181016121934001 심재철   1  
08200101.20181023184036007 정유섭   1  

[7208 rows x 1 columns]

### Insert to DB

In [125]:
with MyMongo() as db:
    db.delete_and_insert_df('audit_quote', 'a1_all', quote.loc[idx_not_empty])

<--Mongo Connected.
{}
Deleted rows: 0
Inserted rows: 7440
Mongo Connection Closed.-->


In [126]:
requests_error

[7809]